In [ ]:
# import sys
# 
# import bertopic
# import tqdm
# !{sys.executable} -m pip install stanza --upgrade
# !{sys.executable} -m pip install torch --upgrade
# 
# import matplotlib.pyplot as plt
# from scipy.cluster.hierarchy import dendrogram, linkage
# 
# def plot_dendrogram(embeddings, labels):
#     linked = linkage(embeddings, 'ward')
#     plt.figure(figsize=(12, 6))
#     dendrogram(linked, labels=labels, leaf_rotation=90)
#     plt.show()
# 
# import pandas as pd
# import ast
# import re
# from sentence_transformers import SentenceTransformer
# from sklearn.cluster import AgglomerativeClustering
# from tqdm import tqdm
# 
# tqdm.pandas()  # Enables df.progress_apply
# 
# from nltk.stem import PorterStemmer
# from nltk.corpus import stopwords
# import nltk
# nltk.download("stopwords")
# 
# stop_words = set(stopwords.words("english"))
# stemmer = PorterStemmer()
# 
# def normalize_text(text):
#     words = re.findall(r'[a-z0-9]+', text.lower())
#     return " ".join(stemmer.stem(w) for w in words if w not in stop_words)
# 
# 
# def extract_unique_amenities(df):
#     unique = set()
#     parsed_amenities = []
#     
#     for val in tqdm(df["amenities"], desc="Parsing amenities"):
#         try:
#             lst = ast.literal_eval(val) if isinstance(val, str) else []
#             lst = [normalize_text(str(a)) for a in lst]
#         except Exception:
#             lst = []
#         parsed_amenities.append(lst)
#         unique.update(lst)
#     print(len(unique))
#     
#     return list(unique), parsed_amenities
# 
# from collections import defaultdict
# 
# def cluster_amenities(unique_amenities, n_clusters=None):
#     print(f"Encoding {len(unique_amenities)} unique amenities...")
#     
# 
#     model = SentenceTransformer("all-MiniLM-L6-v2")
# 
#     embeddings = list(tqdm(model.encode(unique_amenities, show_progress_bar=True), 
#                             total=len(unique_amenities), 
#                             desc="Encoding embeddings"))
#     
#     if not n_clusters:
#         n_clusters = max(2, int(len(unique_amenities) ** 0.5))
#     
#     print(f"Clustering into {n_clusters} groups...")
#     clustering = AgglomerativeClustering(n_clusters=n_clusters)
#     labels = clustering.fit_predict(embeddings)
#     
#     # Group amenities by cluster
#     cluster_to_amenities = defaultdict(list)
#     for amenity, label in zip(unique_amenities, labels):
#         cluster_to_amenities[label].append(amenity)
#     
#     # Pick a representative name for each cluster
#     cluster_names = {}
#     for cluster, items in cluster_to_amenities.items():
#         # Pick the shortest name (after sorting)
#         rep_name = sorted(items, key=len)[0]
#         # Clean name for column use
#         rep_name = rep_name.replace(" ", "_").replace("-", "_")
#         cluster_names[cluster] = rep_name
#     print(cluster_names)
#     
#     # Map amenity → cluster name
#     amenity_to_cluster = {amenity: cluster_names[label] 
#                           for amenity, label in zip(unique_amenities, labels)}
#     plot_dendrogram(embeddings, unique_amenities)
# 
#     return amenity_to_cluster, cluster_to_amenities
# 
# 
# def expand_amenities_semantic(df, amenity_to_cluster, parsed_amenities):
#     cluster_names = set(amenity_to_cluster.values())
#     
#     # Initialize binary columns for each cluster name
#     for cname in cluster_names:
#         df[f"amenity_{cname}"] = 0
#     
#     for idx, lst in tqdm(enumerate(parsed_amenities), 
#                          total=len(parsed_amenities), 
#                          desc="Assigning amenities to clusters"):
#         cluster_ids = {amenity_to_cluster[a] for a in lst if a in amenity_to_cluster}
#         for cname in cluster_ids:
#             df.at[idx, f"amenity_{cname}"] = 1
#     
#     return df
# 
# def process_airbnb_with_semantic_amenities(path, n_clusters=None):
#     df = pd.read_csv(path)
#     unique_amenities, parsed_amenities = extract_unique_amenities(df)
#     print(type(unique_amenities))
#     amenity_to_cluster, cluster_to_amenities = cluster_amenities(unique_amenities, n_clusters)
#     df = expand_amenities_semantic(df, amenity_to_cluster, parsed_amenities)
#     return df, cluster_to_amenities
# 
# 
# if __name__ == "__main__":
#     df, mapping = process_airbnb_with_semantic_amenities(r"C:\Users\hodos\Documents\Uni\Uni-Year-3\Semester2\Data\cleaned_listings_amsterdam.csv")
#     # df.to_csv("cleaned_with_clusters.csv", index=False)
#     # print("Cluster mapping:", mapping)
# 
# df.head()


In [27]:
import pandas as pd
import re
import ast
from sklearn.feature_extraction.text import CountVectorizer
# Import the standard English stop words list from scikit-learn
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from tqdm import tqdm
from collections import defaultdict

# --- Setup: Define Stop Words ---
# For efficiency, create the full set of stop words once.
# We add your custom words to the standard list.
CUSTOM_STOP_WORDS = set(ENGLISH_STOP_WORDS).union(['listing', 'available'])

# Initialize tqdm for pandas functions
tqdm.pandas(desc="Processing DataFrame")

def clean_amenity_text(text):
    """
    Cleans a single amenity string using a comprehensive multi-step process.
    """
    if not isinstance(text, str): return ""

    # 1. Convert to lowercase
    text = text.lower()

    # 2. Remove specific unwanted patterns
    text = re.sub(r'\brequest\b', '', text)
    text = re.sub(r'\d+\s+years?\s+old', '', text)
    text = re.sub(r'ages?\s+\d+', '', text)

    # 3. Keep only lowercase English letters (a-z) and spaces
    text = re.sub(r'[^a-z\s]', '', text)
    
    # 4. Remove all stop words and custom words
    words = text.split()
    filtered_words = [word for word in words if word not in CUSTOM_STOP_WORDS]
    text = ' '.join(filtered_words)

    # 5. Consolidate multiple spaces and strip leading/trailing whitespace
    return re.sub(r'\s+', ' ', text).strip()

def refine_and_group_amenities(candidates):
    """
    Takes messy candidates and applies a two-stage refinement:
    1. Prunes incomplete sub-phrases.
    2. Groups the remaining amenities using a synonym dictionary.
    """
    # --- Stage 1: Prune incomplete fragments (same as before) ---
    print(f"--> Pruning {len(candidates)} initial candidates...")
    candidates.sort(key=len, reverse=True)
    pruned_amenities = []
    temp_set = set(candidates)
    # This logic keeps the longest phrase and discards any of its substrings
    for candidate in candidates:
        if candidate in temp_set:
            # Create a copy of the set to iterate over while modifying the original
            substrings_to_discard = {other for other in temp_set if candidate != other and other in candidate}
            temp_set.difference_update(substrings_to_discard)
            
    pruned_amenities = sorted(list(temp_set), key=len, reverse=True)
    print(f"--> After pruning, {len(pruned_amenities)} candidates remain.")

    # --- Stage 2: Group by Synonyms ---
    # This dictionary gives you full control over grouping.
    # Key = final column name, Value = list of search terms/synonyms.
    SYNONYM_GROUPS = {
        'air_conditioning': ['air conditioning', 'ac', 'aircon'],
        'closet':           ['closet', 'wardrobe','dresser'],
        'tv':               ['tv', 'hdtv', 'television', 'hbo', 'cable', 'netflix', 'hulu', 'amazon prime', 'disney'],
        'wifi':             ['wifi', 'wireless internet', 'internet', 'ethernet'],
        'kitchen':          ['kitchen'],
        'coffee_maker':     ['coffee', 'coffee maker', 'nespresso', 'keurig'],
        'heating':          ['heating', 'heater'],
        'parking':          ['parking'],
        'pool':             ['pool'],
        'hot_tub':          ['hot tub', 'jacuzzi'],
        'bath':             ['bath', 'bathtub'],
        'patio_balcony':    ['patio', 'balcony'],
        'gym':              ['gym', 'fitness'],
        'washer':           ['washer', 'washing machine'],
        'dryer':            ['dryer'],
        'first_aid':        ['first aid'],
        'smoke_alarm':      ['smoke alarm', 'smoke detector'],
        'carbon_monoxide_alarm': ['carbon monoxide alarm', 'carbon monoxide detector'],
        'fire_extinguisher':['fire extinguisher'],
        'refrigerator':     ['refrigerator', 'fridge'],
        'microwave':        ['microwave'],
        'oven':             ['oven'],
        'stove':            ['stove'],
        'books':            ['books'],
        'river':            ['river', 'canal'],
        'shampoo':          ['shampoo'],
        'conditioner':      ['conditioner'],
        'soap':             ['soap', 'body soap']
    }

    groups = defaultdict(list)
    unassigned = []

    for amenity in pruned_amenities:
        was_grouped = False
        # For each amenity, check which synonym group it belongs to
        for standard_name, synonyms in SYNONYM_GROUPS.items():
            # Use regex with word boundaries (\b) to match whole words
            for synonym in synonyms:
                if re.search(r'\b' + re.escape(synonym) + r'\b', amenity):
                    groups[standard_name].append(amenity)
                    was_grouped = True
                    break  # Stop checking other synonyms for this group
            if was_grouped:
                break  # Stop checking other groups for this amenity
        
        if not was_grouped:
            unassigned.append(amenity)

    # For amenities that weren't grouped, they become their own group
    for amenity in unassigned:
        groups[amenity].append(amenity)
        
    print(f"--> Consolidated into {len(groups)} final amenity groups.")
    return groups

def standardize_amenities_final(file_path, min_df_threshold=50):
    print(f"Attempting to load data from: {file_path}")
    try:
        df = pd.read_csv(file_path, engine='python', on_bad_lines='warn')
        print("✓ CSV file loaded successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

    # --- Step 1: Parse Amenities ---
    def parse_amenity_string(s):
        if isinstance(s, str) and s.strip().startswith('['):
            try: return ast.literal_eval(s.strip())
            except: return []
        return []
    df['parsed_amenities'] = df['amenities'].progress_apply(parse_amenity_string)

    # --- Step 2: Discover Candidates ---
    corpus = [clean_amenity_text(a) for l in df['parsed_amenities'] for a in l if clean_amenity_text(a)]
    vectorizer = CountVectorizer(ngram_range=(1, 4), stop_words='english', min_df=min_df_threshold)
    vectorizer.fit(corpus)
    initial_candidates = vectorizer.get_feature_names_out().tolist()

    # --- Step 3: Prune and Group Candidates using Synonym Logic ---
    final_amenity_groups = refine_and_group_amenities(initial_candidates)
    print(final_amenity_groups)
    
    # --- Step 4: Create Binary Columns from Final Groups ---
    print("Creating binary columns from final groups...")
    df['cleaned_amenities_text'] = df['parsed_amenities'].progress_apply(
        lambda lst: ' | '.join([clean_amenity_text(item) for item in lst])
    )

    for group_name, search_terms in tqdm(final_amenity_groups.items(), desc="Creating Columns"):
        # The column name is now the standardized key from the dictionary
        column_name = f"has_{group_name.replace(' ', '_')}"
        # Create a regex 'OR' pattern for all terms in the group
        pattern = '|'.join([r'\b' + re.escape(term) + r'\b' for term in search_terms])
        df[column_name] = df['cleaned_amenities_text'].str.contains(pattern, na=False).astype(int)

    df = df.drop(columns=['parsed_amenities', 'cleaned_amenities_text'])
    print("✓ Binary columns created successfully.")
    return df


   

In [28]:

csv_file_path = r"C:\Users\hodos\Documents\Uni\Uni-Year-3\Semester2\Data\cleaned_listings_amsterdam.csv"
MIN_DF_THRESHOLD = 100 


transformed_df = standardize_amenities_final(
    csv_file_path, min_df_threshold=MIN_DF_THRESHOLD
)

if transformed_df is not None:
    print("\nTransformation complete. Here's a preview:")
    amenity_cols = sorted([col for col in transformed_df.columns if col.startswith('has_')])
    display_cols = ['id', 'name'] + amenity_cols
    
    if len(display_cols) > 20:
        print(f"(Showing a subset of the {len(amenity_cols)} new amenity columns)")
        display_cols = display_cols[:20]

    print(transformed_df[display_cols].head())

Attempting to load data from: C:\Users\hodos\Documents\Uni\Uni-Year-3\Semester2\Data\cleaned_listings_amsterdam.csv
✓ CSV file loaded successfully.


Processing DataFrame: 100%|██████████| 10168/10168 [00:00<00:00, 15226.96it/s]


--> Pruning 575 initial candidates...
--> After pruning, 173 candidates remain.
--> Consolidated into 125 final amenity groups.
defaultdict(<class 'list'>, {'closet': ['clothing storage closet wardrobe', 'storage closet wardrobe dresser', 'clothing storage closet dresser', 'clothing storage walkin closet', 'clothing storage wardrobe', 'clothing storage dresser'], 'coffee_maker': ['coffee maker espresso machine', 'coffee maker pourover coffee', 'drip coffee maker espresso', 'coffee maker french press', 'coffee maker drip coffee', 'maker drip coffee maker', 'coffee maker nespresso', 'machine nespresso'], 'parking': ['paid parking garage premises', 'paid street parking premises', 'paid parking lot premises', 'free parking premises', 'paid parking premises', 'free street parking'], 'oven': ['stainless steel single oven', 'stainless steel oven', 'double oven'], 'air_conditioning': ['portable air conditioning', 'central air conditioning', 'ac split type ductless'], 'stove': ['stainless steel

Creating Columns:  76%|███████▌  | 95/125 [00:09<00:03,  7.73it/s]C:\Users\hodos\AppData\Local\Temp\ipykernel_8152\2759465882.py:162: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = df['cleaned_amenities_text'].str.contains(pattern, na=False).astype(int)
C:\Users\hodos\AppData\Local\Temp\ipykernel_8152\2759465882.py:162: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = df['cleaned_amenities_text'].str.contains(pattern, na=False).astype(int)
Creating Columns:  78%|███████▊  | 97/125 [00:09<00:03,  9

✓ Binary columns created successfully.

Transformation complete. Here's a preview:
(Showing a subset of the 126 new amenity columns)
      id                                               name  has_aeg  \
0  27886  Romantic, stylish B&B houseboat in canal district        0   
1  28871                            Comfortable double room        0   
2  29051                   Comfortable single / double room        0   
3  44391    Quiet 2-bedroom Amsterdam city centre apartment        0   
4  47061                   Charming apartment in old centre        0   

   has_aid_kit  has_air_conditioning  has_availability  has_baby_monitor  \
0            0                     0                 1                 0   
1            0                     0                 1                 0   
2            0                     0                 1                 0   
3            0                     0                 1                 0   
4            1                     0                 1